In [10]:
import numpy as np
import librosa


In [102]:
# EMG Data Shape Info: Shape info: (10319,41)
""" The EMG Data was measured at 2048 Hz, therefore with an estimated video length of 5 seconds, we would see roughly 10240 samples.
    Therefore, we can safely assume that the data presented is the EMG Signals collected from 40 channels every 0.00048828125 seconds and an additional channel
    pull high only at the start of each utterance
"""
emg_data = np.load('Spk1_Block1-Initial_0001_emg.npy')
"""
audio_file = 'Spk1_Block1-Initial_0001_audio.wav'
sr = 16000
audio_data, sr = librosa.load(audio_file, sr=sr)
"""

emg_signals = emg_data[:, :-1]
print(emg_signals.shape)
# This one signifies when a user speaks and doesn't
marker_channel = emg_data[:, -1]
print(marker_channel)
window_size = 100
hop_length = 55
"""
audio_hop_length = int((audio_data.shape[0] / sr) * sr / 63)
n_mfcc = 13

audio_mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=n_mfcc, hop_length=audio_hop_length)
"""

(10319, 40)
[0. 0. 0. ... 0. 0. 0.]


'\naudio_hop_length = int((audio_data.shape[0] / sr) * sr / 63)\nn_mfcc = 13\n\naudio_mfccs = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=n_mfcc, hop_length=audio_hop_length)\n'

In [103]:
def extract_emg_features(emg_signal):
  # Mean absolute value
  mav = np.mean(np.abs(emg_signal))
  # root mean square\
  rms = np.sqrt(np.mean(emg_signal ** 2))
  return mav, rms

emg_features = []

# Process each channel
for ch in range(emg_signals.shape[1]):
  channel_features = []
  for i in range(0, emg_signals.shape[0] - window_size, hop_length):
    window = emg_signals[i:i + window_size, ch]
    mav, rms = extract_emg_features(window) 
    channel_features.append([mav, rms])
  emg_features.append(channel_features)

emg_features = np.array(emg_features)
print(emg_features.shape)

(40, 171, 2)


In [104]:
# Generate labels based on marker channel
labels = []
for i in range(0, marker_channel.shape[0] - window_size, hop_length):
    window_marker = marker_channel[i:i + window_size]
    label = 1 if np.mean(window_marker) > 0 else 0  # Threshold at 0
    labels.append(label)
labels = np.array(labels)



In [105]:
n_samples = emg_features.shape[1]
emg_features_flat = emg_features.reshape(-1, emg_features.shape[2] * emg_features.shape[0])
labels_flat = labels[:n_samples]  # Align labels with features
print(labels_flat.shape)
print (emg_features_flat.shape)

(171,)
(171, 80)


In [106]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(emg_features_flat, labels_flat, test_size=0.2, random_state=42)

In [107]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train the classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Evaluate the model
y_pred = clf.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.9142857142857143
              precision    recall  f1-score   support

           0       1.00      0.25      0.40         4
           1       0.91      1.00      0.95        31

    accuracy                           0.91        35
   macro avg       0.96      0.62      0.68        35
weighted avg       0.92      0.91      0.89        35

